In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sqlite3

conn = sqlite3.connect('../data/MundialF122.db')
cursor = conn.cursor()

In [2]:
lineups = pd.read_sql("SELECT * FROM Lineups;", conn).set_index('Id')
drivers = pd.read_sql("SELECT * FROM Drivers;", conn).set_index('Id')
teams = pd.read_sql("SELECT * FROM Teams;", conn).set_index('Id')

driversId = drivers.index.unique()

lineups_season3 = lineups[lineups['SeasonId'] == 10]

In [3]:
teams

,Name,FullName,UniqueName,Abbreviation,CarId,NationId,Position,ColorRaw,Badge,Prestige,...,PrevTeamId,CreatedAt,UpdatedAt,UserCreated,UserUpdated,SecondaryColorRaw,AliasesRaw,ColorString,SecondaryColorString,TertiaryColorString
Id,,,,,,,,,,,,,,,,,,,,,
1,Alfa Romeo,Alfa Romeo F1 Team Orlen,alfa.romeo.2022,AR,1,209,9,37119,None,30,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FF900000,#FFFFFFFF,None
2,AlphaTauri,Scuderia AlphaTauri,alphatauri.2022,AT,2,108,6,1648700415,None,20,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FF2B4562,#FFFFFFFF,None
3,Alpine,BWT Alpine F1 Team,alpine.2022,ALP,3,77,5,-7339777,None,50,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FF0090FF,#FFFFFFFF,None
4,Aston Martin,Aston Martin Aramco Cognizant F1 Team,aston.martin.2022,AMR,4,227,7,1651441919,None,40,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FF006F62,#FFFFFFFF,None
5,Ferrari,Scuderia Ferrari,ferrari.2022,FER,5,108,3,56575,None,100,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FFDC0000,#FFFFFFFF,None
6,Haas,Haas F1 Team,haas.2022,HAS,6,229,10,-1,None,10,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FFFFFFFF,#FFFFFFFF,None
7,McLaren,McLaren F1 Team,mclaren.2022,MCL,7,227,4,8912895,None,70,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FFFF8700,#FFFFFFFF,None
8,Mercedes,Mercedes-AMG Petronas F1 Team,mercedes.2022,MER,8,83,1,-1093533441,None,90,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FF00D2BE,#FFFFFFFF,None
9,Red Bull,Oracle Red Bull Racing,red.bull.2022,RB,9,14,2,-285210881,None,80,...,None,1665092941,1748285255,1603032504,1603032504,-1,None,#FF0600EF,#FFFFFFFF,None


In [4]:
driversTeams = drivers[['Name']].merge(lineups_season3[['DriverId','TeamId']], left_on='Id', right_on='DriverId', how='left').merge(teams[['Name', 'FullName']], left_on='TeamId',right_on='Id')

In [5]:
def getTrackName(f):
    nombre = os.path.splitext(f)[0]
    return nombre.split('_')[-2]
ruta = '../data/Batallas/2024-25'
archivos = os.listdir(ruta)


In [6]:
battles = pd.concat([
    pd.read_csv(os.path.join(ruta, f)).assign(Circuito = getTrackName(f)) for f in archivos
], ignore_index=True)

battles['Duelo'] = battles.apply(lambda fila: tuple(sorted([fila['DriverA'], fila['DriverB']])), axis = 1)
battles['GanaPiloto1'] = battles.apply(lambda fila: not((fila['DriverB'] == fila['Duelo'][0]) ^ fila['IsWon']), axis = 1)
battles = battles.merge(driversTeams, left_on = 'DriverA', right_on= 'Name_x', how='left')
battles.rename(columns={'Name_y': 'EquipoA'}, inplace=True)
battles.drop(['DriverId', 'Name_x', 'TeamId', 'FullName'], axis=1, inplace=True)
battles = battles.merge(driversTeams, left_on = 'DriverB', right_on= 'Name_x', how='left')
battles.rename(columns={'Name_y': 'EquipoB'}, inplace=True)
battles.drop(['DriverId', 'Name_x', 'TeamId', 'FullName'], axis=1, inplace=True)

battles_inteam = battles[battles['EquipoA'] == battles['EquipoB']]

battles

,DriverA,DriverB,LapStart,LapEnd,TotalDuration,HotDuration,IsWon,TyresA,TyresWearA,TyresB,TyresWearB,Points,Circuito,Duelo,GanaPiloto1,EquipoA,EquipoB
0,Pau Cantero,Abel Romero,10,12,137,121,True,soft,9,medium,54,386.699430,Países Bajos,"(Abel Romero, Pau Cantero)",True,Alpine,Sauber
1,Abel Romero,Eric Reig,4,7,289,59,False,soft,25,soft,22,13.576199,Países Bajos,"(Abel Romero, Eric Reig)",True,Sauber,Aston Martin
2,Adrián Torres,Eric Reig,4,7,289,289,False,soft,25,soft,25,550.911870,Países Bajos,"(Adrián Torres, Eric Reig)",True,Haas,Aston Martin
3,Pau Cantero,Eric Reig,10,13,157,99,True,soft,10,medium,55,155.143000,Países Bajos,"(Eric Reig, Pau Cantero)",True,Alpine,Aston Martin
4,Abel Romero,Eric Reig,16,29,879,866,True,medium,45,medium,45,1895.567700,Países Bajos,"(Abel Romero, Eric Reig)",False,Sauber,Aston Martin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,Víctor Font,Sergio C. Pascual,4,5,50,22,True,medium,16,medium,16,116.049350,Losail,"(Sergio C. Pascual, Víctor Font)",True,NaN,Alpine
4042,Joan Muñoz,Víctor Font,1,1,35,20,False,medium,3,medium,2,17.336872,Losail,"(Joan Muñoz, Víctor Font)",True,Red Bull,NaN
4043,Adrián Torres,Víctor Font,1,1,35,20,False,medium,3,medium,3,41.168470,Losail,"(Adrián Torres, Víctor Font)",True,Haas,NaN
4044,Sergio C. Pascual,Víctor Font,5,5,42,23,False,medium,18,medium,18,43.581028,Losail,"(Sergio C. Pascual, Víctor Font)",True,Alpine,NaN


In [19]:
battles.groupby(['Duelo', 'Circuito']).agg({
        'TotalDuration': ['sum'],
    # 'HotDuration': ['sum', 'mean'],
    # 'IsWon' : 'sum',
    'Points': ['sum', 'mean'],
    # 'DriverA': 'count'
}).sort_values(by = ('TotalDuration','sum'), ascending=False)[:20]

TotalDuration       Points  \
                                                     sum          sum   
Duelo                         Circuito                                  
(Eric Reig, Ferran Leiva)     Reino Unido           4790  9100.176820   
(David Cordón, Josep Garcia)  Bahrain               2594  4321.544523   
(Abel Romero, Adrián Simón)   Reino Unido           2362  1572.041716   
(Eric Garcia, Kevin Leal)     Reino Unido           2361  5983.931346   
(Eric Garcia, Josep Garcia)   Bahrain               2309  2963.051167   
(David Cordón, Eric Garcia)   Bahrain               2138  2145.659533   
(Adrián Torres, Josep Garcia) Bahrain               2103  4331.339570   
(Adrián Simón, Eric Reig)     Reino Unido           2081  2756.491100   
(Adrián Torres, David Cordón) Bahrain               2043  4328.195580   
(David Cordón, Josep Garcia)  México                2036  2812.501276   
(Eric Garcia, Luís Ramos)     Reino Unido           1972  3928.955630   
(Abel Romero, Adrián Torres)  Países Bajos          1939   919.851652   
(Abel Romero, Eric Reig)      Arabia Saudí          1920  5257.949170   
                              México                1884  3894.132022   
(David Cordón, Pau Cantero)   Bélgica               1869  3857.986430   
(Adrián Torres, Eric Reig)    Australia             1852  3582.126632   
(Kevin Leal, Luís Ramos)      Reino Unido           1810  2508.250980   
(David Cordón, Josep Garcia)  Losail                1796  3222.895080   
(Adrián Torres, Eric Reig)    Países Bajos          1771  3612.982170   
(Adrián Torres, Ferran Leiva) Reino Unido           1707  2622.606700   

                                                         
                                                   mean  
Duelo                         Circuito                   
(Eric Reig, Ferran Leiva)     Reino Unido   1137.522102  
(David Cordón, Josep Garcia)  Bahrain        617.363503  
(Abel Romero, Adrián Simón)   Reino Unido    786.020858  
(Eric Garcia, Kevin Leal)     Reino Unido    543.993759  
(Eric Garcia, Josep Garcia)   Bahrain        740.762792  
(David Cordón, Eric Garcia)   Bahrain        238.406615  
(Adrián Torres, Josep Garcia) Bahrain        481.259952  
(Adrián Simón, Eric Reig)     Reino Unido   1378.245550  
(Adrián Torres, David Cordón) Bahrain        618.313654  
(David Cordón, Josep Garcia)  México         281.250128  
(Eric Garcia, Luís Ramos)     Reino Unido    785.791126  
(Abel Romero, Adrián Torres)  Países Bajos   306.617217  
(Abel Romero, Eric Reig)      Arabia Saudí   876.324862  
                              México         432.681336  
(David Cordón, Pau Cantero)   Bélgica        428.665159  
(Adrián Torres, Eric Reig)    Australia      895.531658  
(Kevin Leal, Luís Ramos)      Reino Unido    278.694553  
(David Cordón, Josep Garcia)  Losail         402.861885  
(Adrián Torres, Eric Reig)    Países Bajos  1806.491085  
(Adrián Torres, Ferran Leiva) Reino Unido   2622.606700

In [14]:
battles[battles['Circuito'] == 'Bahrain'].groupby(['Duelo']).agg({
    #     'TotalDuration': ['sum'],
    # # 'HotDuration': ['sum', 'mean'],
    # # 'IsWon' : 'sum',
    'Points': ['sum', 'mean'],
    # 'DriverA': 'count'
}).sort_values(by = ('Points','sum'), ascending=False)


Points            
                                              sum        mean
Duelo                                                        
(Adrián Torres, Josep Garcia)         4331.339570  481.259952
(Adrián Torres, David Cordón)         4328.195580  618.313654
(David Cordón, Josep Garcia)          4321.544523  617.363503
(Eric Garcia, Josep Garcia)           2963.051167  740.762792
(David Cordón, Eric Garcia)           2145.659533  238.406615
(Eric Garcia, Iván Fajardo)           1483.820060  494.606687
(Adrián Torres, Pau Cantero)          1437.640354  359.410089
(Iván Fajardo, Josep Garcia)          1386.976992  346.744248
(Adrián Torres, Eric Garcia)          1184.682307  236.936461
(Joan Muñoz, Pau Cantero)             1135.778700  227.155740
(Roberto Floria, Sergio C. Pascual)    810.723400  405.361700
(David Cordón, Pau Cantero)            645.772500  645.772500
(Luís Ramos, Pau Cantero)              638.181370  212.727123
(David Cordón, Iván Fajardo)           635.790674  158.947668
(Eric Reig, Iván Fajardo)              552.866000  184.288667
(Kevin Leal, Toni Casado)              551.324395  110.264879
(Adrià Batanés, Eric Reig)             507.974940  253.987470
(Adrián Torres, Iván Fajardo)          402.583920  201.291960
(Ferran Leiva, Kevin Leal)             402.550100  402.550100
(Kevin Leal, Víctor Font)              393.568466  196.784233
(Eudald Clascà, Roberto Ascenso)       325.494200   81.373550
(Pau Cantero, Roberto Floria)          321.675920  160.837960
(Ferran Leiva, Roberto Ascenso)        289.504890  144.752445
(Ferran Leiva, Sergio C. Pascual)      237.893500  118.946750
(Iván Fajardo, Luís Ramos)             220.411270  220.411270
(Roberto Ascenso, Víctor Font)         194.455690  194.455690
(Abel Romero, Iván Fajardo)            193.586530   96.793265
(Roberto Ascenso, Roberto Floria)      167.999590  167.999590
(Eudald Clascà, Sergio C. Pascual)     163.239906   81.619953
(Abel Romero, David Cordón)            144.392640  144.392640
(Toni Casado, Víctor Font)             142.754040  142.754040
(Eudald Clascà, Kevin Leal)            127.987160  127.987160
(Roberto Ascenso, Sergio C. Pascual)   127.958370  127.958370
(Abel Romero, Josep Garcia)             67.743880   67.743880
(Abel Romero, Joan Muñoz)               47.516983   47.516983
(David Cordón, Luís Ramos)              47.250557   47.250557
(Adrià Batanés, Iván Fajardo)           46.356730   46.356730
(Eudald Clascà, Víctor Font)            46.263638   46.263638
(Sergio C. Pascual, Víctor Font)        42.605705   42.605705
(David Cordón, Eric Reig)               24.855734   24.855734
(Eric Garcia, Eric Reig)                16.405497   16.405497
(Josep Garcia, Pau Cantero)             13.169672    6.584836
(Abel Romero, Pau Cantero)               4.285890    4.285890

In [65]:
battles_h2h = battles.groupby(['DriverA', 'DriverB']).agg({
    'TotalDuration': ['sum', 'mean'],
    'HotDuration': ['sum', 'mean'],
    'IsWon' : 'sum',
    'Points': ['sum', 'mean'],
    'DriverA': 'count'
})
battles_h2h=battles_h2h.rename(columns={'DriverA': 'Count'}).reset_index()
battles_h2h.sort_values(by = ('TotalDuration', 'sum'), ascending = False)

DriverA         DriverB TotalDuration             HotDuration  \
                                              sum        mean         sum   
7       Abel Romero       Eric Reig          9913  254.179487        7583   
166       Eric Reig   Adrián Torres          7717  233.848485        6394   
165       Eric Reig    Adrián Simón          7453  225.848485        5525   
2       Abel Romero   Adrián Torres          7308  270.666667        5548   
121    David Cordón    Josep Garcia          5969  221.074074        3738   
..              ...             ...           ...         ...         ...   
258      Joan Muñoz   Eudald Clascà            39   39.000000          21   
506  Álvaro Teixidó     Víctor Font            35   35.000000          32   
493  Álvaro Teixidó   Eudald Clascà            33   33.000000          13   
503  Álvaro Teixidó  Roberto Floria            31   31.000000          19   
468     Víctor Font  Arnau Valverde            31   31.000000          31   

                IsWon        Points             Count  
           mean   sum           sum        mean count  
7    194.435897    22  19552.874998  501.355769    39  
166  193.757576     7  13431.346361  407.010496    33  
165  167.424242     5   9656.356883  292.616875    33  
2    205.481481     9   9152.782068  338.991928    27  
121  138.444444    14   9404.199278  348.303677    27  
..          ...   ...           ...         ...   ...  
258   21.000000     0     52.467000   52.467000     1  
506   32.000000     0     51.806600   51.806600     1  
493   13.000000     0     38.880333   38.880333     1  
503   19.000000     0     57.976340   57.976340     1  
468   31.000000     0     67.439545   67.439545     1  

[507 rows x 10 columns]

In [ ]:
battles_h2h_total = battles_inteam.groupby(['Duelo']).agg({
    'TotalDuration': ['sum', 'mean'],
    'HotDuration': ['sum', 'mean'],
    'IsWon' : 'count',
    'Points': ['sum', 'mean'],
    'DriverA': 'count',
    'GanaPiloto1': 'sum'
})
battles_h2h.rename(columns = {('DriverA','count'): 'Count'})
battles_h2h_total.sort_values(by = ('TotalDuration', 'sum'), ascending = False)[0:20]

In [78]:
battles_h2h_total = battles_inteam.groupby(['Duelo']).agg({
    'TotalDuration': 'sum',
    'HotDuration': 'sum',
    # 'IsWon' : 'count',
    'Points': 'sum',
    'DriverA': 'count',
    'GanaPiloto1': 'sum'
})
battles_h2h_total.rename(columns = {('DriverA'): 'Duelos'}, inplace=True)
battles_h2h_total.sort_values(by = ('TotalDuration'), ascending = False)[0:20]

,TotalDuration,HotDuration,Points,Duelos,GanaPiloto1
Duelo,,,,,
"(David Cordón, Iván Fajardo)",3950,2541,5655.698987,32,23
"(Joan Muñoz, Josep Garcia)",3516,1879,4378.878788,30,15
"(Adrià Batanés, Ferran Leiva)",3379,2022,4479.381237,26,4
"(Adrián Torres, Eric Garcia)",3091,2012,3575.905628,18,12
"(Pau Cantero, Sergio C. Pascual)",2615,1536,2487.089717,24,21
"(Adrián Simón, Roberto Ascenso)",1100,654,1903.055658,15,11
"(Jorge Arnal, Roberto Floria)",459,256,920.819555,7,6
"(Eric Reig, Kevin Leal)",312,160,306.901943,4,4
"(Arnau Valverde, Eudald Clascà)",83,53,410.240610,2,1


In [64]:
s = battles_h2h_total[('TotalDuration','sum')].sort_values(ascending = False)[0:20]
s_index = s.index

for i in range(len(s)):
    total_s = s.iloc[i]
    horas = total_s // 3600
    minutos = (total_s % 3600) // 60
    segundos = total_s % 60
    if i == 0:
        print(i+1, s_index[i],'\t\t', f"{horas}h {minutos}m {segundos}s")
    else:
        print(i+1, s_index[i],'\t', f"{horas}h {minutos}m {segundos}s")

1 ('Abel Romero', 'Eric Reig') 		 4h 21m 51s
2 ('Adrián Torres', 'Eric Reig') 	 3h 5m 28s
3 ('David Cordón', 'Josep Garcia') 	 3h 4m 50s
4 ('Abel Romero', 'Adrián Torres') 	 2h 54m 0s
5 ('Adrián Simón', 'Adrián Torres') 	 2h 42m 7s
6 ('Adrián Simón', 'Eric Reig') 	 2h 33m 17s
7 ('Iván Fajardo', 'Josep Garcia') 	 2h 12m 13s
8 ('Eric Reig', 'Ferran Leiva') 	 2h 9m 53s
9 ('Joan Muñoz', 'Roberto Ascenso') 	 2h 6m 35s
10 ('Adrián Torres', 'Pau Cantero') 	 1h 58m 52s
11 ('Abel Romero', 'Adrián Simón') 	 1h 54m 29s
12 ('Arnau Valverde', 'Pau Cantero') 	 1h 42m 10s
13 ('Eric Garcia', 'Luís Ramos') 	 1h 39m 43s
14 ('Adrián Simón', 'Ferran Leiva') 	 1h 39m 8s
15 ('Eric Garcia', 'Ferran Leiva') 	 1h 37m 36s
16 ('Eric Garcia', 'Josep Garcia') 	 1h 33m 57s
17 ('Adrián Torres', 'David Cordón') 	 1h 32m 37s
18 ('Joan Muñoz', 'Pau Cantero') 	 1h 32m 13s
19 ('Arnau Valverde', 'Joan Muñoz') 	 1h 30m 38s
20 ('Eric Garcia', 'Kevin Leal') 	 1h 27m 23s


In [10]:
s = battles_h2h_total[('TotalDuration','sum')].sort_values(ascending = False)
s_index = s.index
archienemigos = {}
for i in range(len(s)):
    drivers_i = s_index[i]
    if drivers_i[0] not in archienemigos:
        archienemigos[drivers_i[0]] = drivers_i[1]
    if drivers_i[1] not in archienemigos:
        archienemigos[drivers_i[1]] = drivers_i[0]
archienemigos

{'Abel Romero': 'Eric Reig',
 'Eric Reig': 'Abel Romero',
 'Adrián Torres': 'Eric Reig',
 'David Cordón': 'Josep Garcia',
 'Josep Garcia': 'David Cordón',
 'Adrián Simón': 'Adrián Torres',
 'Iván Fajardo': 'Josep Garcia',
 'Ferran Leiva': 'Eric Reig',
 'Joan Muñoz': 'Roberto Ascenso',
 'Roberto Ascenso': 'Joan Muñoz',
 'Pau Cantero': 'Adrián Torres',
 'Arnau Valverde': 'Pau Cantero',
 'Eric Garcia': 'Luís Ramos',
 'Luís Ramos': 'Eric Garcia',
 'Kevin Leal': 'Eric Garcia',
 'Sergio C. Pascual': 'Roberto Ascenso',
 'Álvaro Teixidó': 'Roberto Ascenso',
 'Adrià Batanés': 'Ferran Leiva',
 'Roberto Floria': 'Kevin Leal',
 'Toni Casado': 'Kevin Leal',
 'Jorge Arnal': 'Ferran Leiva',
 'Eudald Clascà': 'Víctor Font',
 'Víctor Font': 'Eudald Clascà',
 'Dimitry Ptrenko': 'Iván Fajardo'}

In [57]:
s = battles_h2h_total[[('TotalDuration','sum'),('GanaPiloto1','sum'), ('DriverA', 'count')]].sort_values(by = ('TotalDuration','sum'), ascending = False)
s_index = s.index
individual = []
driver = 'Dimitry Ptrenko'
for i in range(len(s)):
    drivers_i = s_index[i]
    total_s = s.iloc[i][('TotalDuration','sum')]
    gana1 = s.iloc[i][('GanaPiloto1','sum')]
    duelos = s.iloc[i][('DriverA','count')]
    horas = total_s // 3600
    minutos = (total_s % 3600) // 60
    segundos = total_s % 60
    if drivers_i[0] == driver:
        individual.append({'Ranking': i+1, 'Enemigo': drivers_i[1], 'Duracion': f"{horas}h {minutos}m {segundos}s", 'Duelos': duelos, 'Gana ' + driver : gana1, 'Gana Rival': duelos-gana1})
    elif drivers_i[1] == driver:
        individual.append({'Ranking': i+1, 'Enemigo': drivers_i[0], 'Duracion': f"{horas}h {minutos}m {segundos}s", 'Duelos': duelos, 'Gana ' + driver: duelos-gana1, 'Gana Rival': gana1 })
individual_df = pd.DataFrame(individual)
# individual_df['Padreo'] = individual_df['Gana Adrián Simón']-individual_df['Gana Rival']
individual_df.sort_values(by = ('Duelos'), ascending = False)

,Ranking,Enemigo,Duracion,Duelos,Gana Dimitry Ptrenko,Gana Rival
0,229,Iván Fajardo,0h 5m 3s,2,0,2
1,235,Arnau Valverde,0h 3m 48s,2,0,2
2,237,Adrián Torres,0h 3m 31s,2,0,2
3,238,Luís Ramos,0h 3m 24s,2,0,2
4,240,Pau Cantero,0h 3m 19s,2,0,2
6,247,Ferran Leiva,0h 2m 37s,2,1,1
8,257,Abel Romero,0h 1m 39s,2,0,2
7,256,Adrián Simón,0h 1m 54s,2,0,2
5,246,Kevin Leal,0h 2m 40s,1,1,0
9,260,Eric Reig,0h 0m 50s,1,0,1


In [ ]:
battles_attack = battles.groupby(['DriverB']).agg({
    'IsWon' : 'sum',
    'DriverB' : 'count',
    'Points' : 'sum',
    'TotalDuration' : 'sum'
})
battles_attack.rename(columns = {'IsWon': 'AtaqueExito', 'DriverB': 'Ataque'}, inplace=True)
battles_attack.sort_values(by = 'AtaqueExito', ascending = False)

,AtaqueExito,Ataque,Points,TotalDuration
DriverB,,,,
Adrián Torres,146,267,88089.071988,44741
Abel Romero,137,226,58041.688378,24462
Eric Reig,118,182,58787.337668,26984
Pau Cantero,110,259,68603.577999,35792
Adrián Simón,98,225,62211.071083,31604
Eric Garcia,98,229,58722.832953,31252
Joan Muñoz,95,220,54349.555322,27566
Roberto Ascenso,89,217,46577.571897,26448
Ferran Leiva,88,201,57217.427739,31532


In [ ]:
battles_defense = battles.groupby(['DriverA']).agg({
    'IsWon' : 'sum',
    'DriverA' : 'count',
    'Points' : 'sum',
    'TotalDuration' : 'sum'
})
battles_defense['DefensaExito'] = battles_defense['DriverA']-battles_defense['IsWon']
battles_defense.rename(columns = {'DriverA': 'Defensa'}, inplace=True)
battles_defense.sort_values(by = 'DefensaExito', ascending = False)

,IsWon,Defensa,Points,TotalDuration,DefensaExito
DriverA,,,,,
Adrián Torres,44,247,45929.242803,31303,203
Abel Romero,44,231,55196.859732,37699,187
Eric Reig,35,212,57539.238489,36523,177
Pau Cantero,88,242,53559.020007,29247,154
Roberto Ascenso,73,224,50743.178869,28342,151
Adrián Simón,35,175,37028.935886,23460,140
Josep Garcia,102,240,52410.521048,29879,138
Iván Fajardo,127,264,64086.959364,30950,137
Ferran Leiva,61,197,44000.237346,26213,136


In [ ]:
battles_driver = pd.merge(battles_attack, battles_defense, left_index=True, right_index=True)
battles_driver['DueloExito'] = battles_driver['AtaqueExito']+battles_driver['DefensaExito']
battles_driver['Points'] = battles_driver['Points_x'] + battles_driver['Points_y']
battles_driver['TotalDuration'] = battles_driver['TotalDuration_x'] + battles_driver['TotalDuration_y']
battles_driver['Duelos'] = battles_driver['Ataque'] + battles_driver['Defensa']
battles_driver['DueloPer'] = battles_driver['DueloExito'] / battles_driver['Duelos']
battles_driver['AtaquePer'] = battles_driver['AtaqueExito'] / battles_driver['Ataque']
battles_driver['DefensaPer'] = battles_driver['DefensaExito'] / battles_driver['Defensa']
battles_driver_final = battles_driver[['Duelos', 'DueloExito', 'DueloPer', 'Ataque', 'AtaqueExito', 'AtaquePer', 'Defensa', 'DefensaExito', 'DefensaPer', 'TotalDuration']]
battles_driver_final.sort_values(by = 'TotalDuration', ascending= False)

,Duelos,DueloExito,DueloPer,Ataque,AtaqueExito,AtaquePer,Defensa,DefensaExito,DefensaPer,TotalDuration
DriverB,,,,,,,,,,
Adrián Torres,514,349,0.678988,267,146,0.546816,247,203,0.821862,76044
Pau Cantero,501,264,0.526946,259,110,0.424710,242,154,0.636364,65039
Josep Garcia,487,223,0.457906,247,85,0.344130,240,138,0.575000,64489
David Cordón,451,197,0.436807,220,70,0.318182,231,127,0.549784,63961
Eric Reig,394,295,0.748731,182,118,0.648352,212,177,0.834906,63507
Eric Garcia,440,232,0.527273,229,98,0.427948,211,134,0.635071,62831
Iván Fajardo,509,187,0.367387,245,50,0.204082,264,137,0.518939,62248
Abel Romero,457,324,0.708972,226,137,0.606195,231,187,0.809524,62161
Ferran Leiva,398,224,0.562814,201,88,0.437811,197,136,0.690355,57745


In [52]:
battles_driver_final.sort_values(by='AtaquePer', ascending=False)

,Duelos,DueloExito,DueloPer,Ataque,AtaqueExito,AtaquePer,Defensa,DefensaExito,DefensaPer,TotalDuration
DriverB,,,,,,,,,,
Eric Reig,394,295,0.748731,182,118,0.648352,212,177,0.834906,63507
Abel Romero,457,324,0.708972,226,137,0.606195,231,187,0.809524,62161
Adrián Torres,514,349,0.678988,267,146,0.546816,247,203,0.821862,76044
Ferran Leiva,398,224,0.562814,201,88,0.437811,197,136,0.690355,57745
Adrián Simón,400,238,0.595000,225,98,0.435556,175,140,0.800000,55064
Joan Muñoz,435,222,0.510345,220,95,0.431818,215,127,0.590698,56810
Eric Garcia,440,232,0.527273,229,98,0.427948,211,134,0.635071,62831
Pau Cantero,501,264,0.526946,259,110,0.424710,242,154,0.636364,65039
Roberto Ascenso,441,240,0.544218,217,89,0.410138,224,151,0.674107,54790


## Season 4

In [4]:
def getTrackName(f):
    nombre = os.path.splitext(f)[0]
    return nombre.split('_')[-2]
ruta = '../data/Batallas/2025-26'
archivos = os.listdir(ruta)

In [ ]:
battles = pd.concat([
    pd.read_csv(os.path.join(ruta, f)).assign(Circuito = getTrackName(f)) for f in archivos
], ignore_index=True)

battles['Duelo'] = battles.apply(lambda fila: tuple(sorted([fila['DriverA'], fila['DriverB']])), axis = 1)
battles['GanaPiloto1'] = battles.apply(lambda fila: not((fila['DriverB'] == fila['Duelo'][0]) ^ fila['IsWon']), axis = 1)
# battles.drop(['DriverId', 'Name_x', 'TeamId', 'FullName'], axis=1, inplace=True)

battles

,DriverA,DriverB,LapStart,LapEnd,TotalDuration,HotDuration,IsWon,TyresA,TyresWearA,TyresB,TyresWearB,Points,Circuito,Duelo,GanaPiloto1
0,Luís Ramos,Adrián Torres,1,1,48,37,True,intermediate,2,intermediate,2,372.391880,Shanghai,"(Adrián Torres, Luís Ramos)",True
1,Eric Reig,Adrián Torres,1,5,395,382,True,intermediate,8,intermediate,8,1011.011500,Shanghai,"(Adrián Torres, Eric Reig)",True
2,Adrián Torres,Eric Reig,5,6,166,46,False,intermediate,17,intermediate,17,142.824970,Shanghai,"(Adrián Torres, Eric Reig)",True
3,Adrián Torres,Adrián Simón,1,2,83,60,False,intermediate,2,intermediate,3,63.379673,Shanghai,"(Adrián Simón, Adrián Torres)",False
4,Luís Ramos,Adrián Simón,1,5,436,390,True,intermediate,7,intermediate,9,747.693700,Shanghai,"(Adrián Simón, Luís Ramos)",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Josep Garcia,Eric Garcia,14,15,64,19,False,intermediate,51,intermediate,3,47.900394,Shanghai,"(Eric Garcia, Josep Garcia)",False
89,Ferran Leiva,Eric Garcia,14,15,56,45,False,intermediate,51,intermediate,4,78.928310,Shanghai,"(Eric Garcia, Ferran Leiva)",False
90,David Cordón,Eric Garcia,15,15,46,29,False,intermediate,52,intermediate,9,50.159153,Shanghai,"(David Cordón, Eric Garcia)",True
91,Sergio C. Pascual,Eric Garcia,15,16,94,35,False,intermediate,54,intermediate,14,82.162180,Shanghai,"(Eric Garcia, Sergio C. Pascual)",False


In [16]:
battles[battles['Circuito'] == 'Shanghai'].sort_values(by = 'HotDuration', ascending=False)

,DriverA,DriverB,LapStart,LapEnd,TotalDuration,HotDuration,IsWon,TyresA,TyresWearA,TyresB,TyresWearB,Points,Circuito,Duelo,GanaPiloto1
49,Pau Cantero,David Cordón,5,12,870,786,True,intermediate,25,intermediate,19,1718.785500,Shanghai,"(David Cordón, Pau Cantero)",True
36,Josep Garcia,Ferran Leiva,14,22,884,767,True,intermediate,19,intermediate,15,1522.466200,Shanghai,"(Ferran Leiva, Josep Garcia)",True
69,Sergio C. Pascual,Pau Cantero,21,28,825,628,False,intermediate,36,intermediate,48,1039.132800,Shanghai,"(Pau Cantero, Sergio C. Pascual)",False
10,Adrián Simón,Joan Muñoz,19,25,716,493,False,intermediate,34,intermediate,33,845.632800,Shanghai,"(Adrián Simón, Joan Muñoz)",True
75,David Cordón,Roberto Ascenso,3,10,789,406,False,intermediate,23,intermediate,19,2676.251700,Shanghai,"(David Cordón, Roberto Ascenso)",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,Josep Garcia,Arnau Valverde,5,6,118,15,False,intermediate,18,intermediate,18,4.964107,Shanghai,"(Arnau Valverde, Josep Garcia)",False
15,Joan Muñoz,Luís Ramos,17,19,165,13,False,intermediate,13,intermediate,18,63.735750,Shanghai,"(Joan Muñoz, Luís Ramos)",True
42,Arnau Valverde,Josep Garcia,21,22,80,13,False,intermediate,28,intermediate,24,36.625270,Shanghai,"(Arnau Valverde, Josep Garcia)",True
29,Arnau Valverde,Ferran Leiva,4,4,43,13,False,intermediate,10,intermediate,12,2.039915,Shanghai,"(Arnau Valverde, Ferran Leiva)",True


In [19]:
battles[battles['Circuito'] == 'Shanghai'].groupby(['Duelo']).agg({
        # 'TotalDuration': ['sum'],
    'HotDuration': 'sum',
    # # 'IsWon' : 'sum',
    # 'Points': ['sum', 'mean'],
    # 'DriverA': 'count'
}).sort_values(by = 'HotDuration', ascending=False)[:10]

,HotDuration
Duelo,
"(David Cordón, Roberto Ascenso)",1044
"(Ferran Leiva, Josep Garcia)",1036
"(Arnau Valverde, Josep Garcia)",961
"(David Cordón, Pau Cantero)",864
"(Adrián Simón, Joan Muñoz)",826
"(David Cordón, Sergio C. Pascual)",792
"(Pau Cantero, Sergio C. Pascual)",717
"(Roberto Ascenso, Sergio C. Pascual)",673
"(Iván Fajardo, Josep Garcia)",568
